<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

In [1]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [15]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [16]:
def model(c):
    c=DotMap(c)
    Ltot = c.Ftot - c.Vtot
    V = c.Vy * c.Vtot
    L = c.Lx * Ltot
    F = c.Fz * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix(0., F, c.FT)
    VH = p.Hmix(V, 0., c.flashT)
    LH = p.Hmix(0., L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
                - c.Vy*c.flashP)
    fugacity = (fugacity/jnp.sum(c.Vy*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [17]:
s=DotMap()
s.Ftot=10
s.Fz = jnp.array([1/3, 1/3, 1/3])
s.FT = 400

s.flashP= 101325
v=DotMap()
v.Vy = vsc.Comp(s.Fz)
v.Lx = vsc.Comp(s.Fz)
v.flashT = vsc.Range(360, 300, s.FT)
v.Vtot = vsc.Range(s.Ftot/2, 0., s.Ftot)

In [18]:
c=vsc.VSC(v,s)
model(c.xtoc(c.x))

DeviceArray(0.06531803, dtype=float64)

In [19]:
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:3, f: 0.06531803353475606
1:5, f: 0.004322823697925632
2:5, f: 0.00017278942431539642
3:5, f: 2.9528721063611253e-06
4:5, f: 2.0437620162466306e-09
Final results:
f: 1.137291061949767e-15
x: [-0.06257052 -0.03792896 -1.53089992  0.29515518  0.19118277  0.11507143]


In [20]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)
df

,,"(vector3, 1)","(vector3, 2)","(vector3, 3)"
FT,400,,,
Ftot,10,,,
Fz,,0.333333,0.333333,0.333333
Lx,,0.323675,0.33175,0.344575
Vtot,1.77862,,,
Vy,,0.377977,0.340651,0.281372
flashP,101325,,,
flashT,352.874,,,
